# Detect spectral peaks in geophone spectrograms

In [1]:
# Imports
from os.path import join

from pandas import concat
from utils_basic import SPECTROGRAM_DIR as indir, GEO_STATIONS as stations
from utils_spec import assemble_spec_filename, read_geo_spectrograms, find_geo_station_spectral_peaks
from utils_plot import plot_geo_total_psd_and_peaks, save_figure

In [2]:
# Inputs
# Data
window_length = 60.0
overlap = 0.0
downsample = True
downsample_factor = 60

# Finding peaks
rbw_threshold = 0.2
prom_threshold = 5
freqmin = None
freqmax = None


In [3]:
# Loop over days and stations
peak_dfs = []
for station in stations:
    print(f"### Working on {station}... ###")
    
    # Read the spectrograms
    print("Reading the spectrograms")
    if downsample:
        filename = assemble_spec_filename("whole_deployment", "day", "geo", station, window_length, overlap, downsample, downsample_factor = downsample_factor)
        inpath = join(indir, filename)
        stream_spec = read_geo_spectrograms(inpath)
    else:
        filename = assemble_spec_filename("whole_deployment", "day", "geo", station, window_length, overlap, downsample)
        inpath = join(indir, filename)
        stream_spec = read_geo_spectrograms(inpath)

    # Loop over blocks
    time_labels = stream_spec.get_time_labels()

    for time_label in time_labels:
        print(f"### Working on {time_label}... ###")
        stream_spec_block = stream_spec.select(time_label = time_label)

        # Find the peaks
        print("Detecting the peaks...")
        peak_df, _ = find_geo_station_spectral_peaks(stream_spec_block, rbw_threshold = rbw_threshold, prom_threshold = prom_threshold, freqmin = freqmin, freqmax = freqmax)
        print(f"In total, {len(peak_df)} spectral peaks found.")

        # Add the station to the dataframe
        peak_df["station"] = station

        # Append to the list
        peak_dfs.append(peak_df)

### Working on A01... ###
Reading the spectrograms
### Working on 20200110000000000000... ###
Detecting the peaks...
In total, 1830 spectral peaks found.
### Working on 20200111000000000000... ###
Detecting the peaks...
In total, 3330 spectral peaks found.
### Working on 20200112000000000000... ###
Detecting the peaks...
In total, 2846 spectral peaks found.
### Working on 20200113000000000000... ###
Detecting the peaks...
In total, 3995 spectral peaks found.
### Working on 20200114000000000000... ###
Detecting the peaks...
In total, 4243 spectral peaks found.
### Working on 20200115000000000000... ###
Detecting the peaks...
In total, 3087 spectral peaks found.
### Working on 20200116000000000000... ###
Detecting the peaks...
In total, 2686 spectral peaks found.
### Working on 20200117000000000000... ###
Detecting the peaks...
In total, 2103 spectral peaks found.
### Working on 20200118000000000000... ###
Detecting the peaks...
In total, 3083 spectral peaks found.
### Working on 2020011

In [4]:
# Save the results
peak_df = concat(peak_dfs)
peak_df.reset_index(drop = True, inplace = True)

print("Saving the results...")
outdir = indir
filename = f"geo_spectral_peaks_window{window_length:.0f}_overlap{overlap:.1f}_prom{prom_threshold:.0f}_rbw{rbw_threshold:.1f}.csv"
outpath = join(outdir, filename)

peak_df.to_csv(outpath)
print(f"Results saved to {outpath}")

Saving the results...
Results saved to /fp/projects01/ec332/data/spectrograms/geo_spectral_peaks_window60_overlap0.0_prom5_rbw0.2.csv
